##### 1.   **Configurações iniciais**

1.1 Informações sobre a GPU

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

1.2 Memória disponível

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

Your runtime has 89.6 gigabytes of available RAM



1.3 Sincronização com o diretório base no Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


1.4 Definição de variáveis globais

In [ ]:
# Diretório base no Drive
DIR_BASE = "/content/drive/My Drive/"

# Tamanho e número de canais da imagem
INPUT_SHAPE = (64, 64, 3)

# Hiperparâmetros de treinamento
BATCH_SIZE = 32
DROPOUT = 0.5
EPOCH = 100
LRN_RATE = 0.001

1.5 Importação de bibliotecas para o ambiente

In [ ]:
# Bibliotecas para desenvolvimento de redes neurais
import tensorflow as tf
from tensorflow import keras

# Bibliotecas do Keras para processamento de imagem
from keras.utils import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator

# Módulos genéricos do Keras para desenvolvimento de Redes Neurais
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Activation
from keras.models import load_model

# Módulos para Otimização
from keras.optimizers import Adam

# Módulos para Regularização
from keras.layers import BatchNormalization, Dropout

# Módulos do Keras específicos para Redes Neurais Convolucionais
from keras.layers import Conv2D, MaxPooling2D, Flatten

# Bibliotecas para análise de dados e operações matriciais
import numpy as np
import pandas as pd

# Bibliotecas para visualização dos dados
import plotly.express as px

# Bibliotecas para avaliação de desempenho
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

##### 2.   **Pré-processamento de dados** 

In [ ]:
train_datagen = ImageDataGenerator(
                    rescale = 1./255,
                    height_shift_range = 0.1,
                    width_shift_range = 0.1)

val_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

2.2 Conjunto de **Treinamento**

In [ ]:
train_data = train_datagen.flow_from_directory(
    DIR_BASE+'Dataset/BSB2023/Treinamento/sequencia',
    target_size = (64, 64),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    shuffle=True,
    seed=42
)

2.3 Conjunto de **Validação**

In [ ]:
val_data = val_datagen.flow_from_directory(
    DIR_BASE+'Dataset/BSB2023/Validacao/sequencia',
    target_size = (64, 64),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    shuffle=True,
    seed=42
)

2.4 Conjunto de **Teste**

In [ ]:
test_data = test_datagen.flow_from_directory(
    DIR_BASE+'Dataset/BSB2023/Teste/sequencia',
    target_size = (64, 64),
    batch_size = 1,
    class_mode = None,
    shuffle=False,
    seed=42
)

2.5 Verificação da **estrutura dos dados** de entrada

In [ ]:
for image_batch, labels_batch in train_data:
  print("Estrutura dos dados de entrada:",image_batch.shape)
  break

print("Classes do problema:",train_data.class_indices)

##### 3.   Desenvolvimento do modelo baseado em **redes neurais convolucionais**

3.1 Definição da classe miniVGGNet

In [ ]:
class Model:
  @staticmethod
  def build(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1

    # CONV => Activation => CONV => Activation => POOL
    model.add(Conv2D(input_shape=INPUT_SHAPE, filters=32, kernel_size=(3, 3), strides=(1, 1), padding="same"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding="same"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(DROPOUT))

    # CONV => Activation => CONV => Activation => POOL
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding="same"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding="same"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(DROPOUT))

    # Dense Layer
    model.add(Flatten())
    model.add(Dense(512))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))
    model.add(Dropout(DROPOUT))

    # Softmax classifier
    model.add(Dense(classes))
    model.add(Activation("softmax"))

    return model

3.2 Instanciar o modelo

In [ ]:
model = miniVGGNet.build(width = 64, height = 64, depth = 3, classes=5)

3.3 Sumarização do modelo

In [ ]:
model.summary()

3.4 Compilação do modelo

In [ ]:
# Otimizador
OPT = Adam(learning_rate=LRN_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-07)

# Compilação
model.compile(optimizer= OPT, loss='categorical_crossentropy', metrics=["accuracy"])

3.4 **Treinamento**

In [ ]:
history = model.fit(train_data, epochs=EPOCH, validation_data=val_data)

# Salva o modelo ao final do processo de treinamento
model.save(DIR_BASE+'Modelos/sequencia_64_64_1e-03_05_100_20230422.h5')

##### 4.   Avaliação de Desempenho

4.1 Importar o modelo previamente treinado

In [ ]:
model = load_model(DIR_BASE+'Modelos/sequencia_64_64_1e-03_05_100_20230422.h5')

4.2 **Predição** das amostras de teste

In [ ]:
test_data.reset()
probabilidades = model.predict(test_data, verbose=1)

4.3 Obtenção dos rótulos preditos e verdadeiros para cada amostra do conjunto de teste

In [ ]:
predicoes = []
num_amostras_teste = 200

for i in range(num_amostras_teste):
  lista = list(probabilidades[i])
  classe_predita = lista.index(max(lista))
  predicoes.append(classe_predita)

predito = np.array(predicoes)

# Definição dos rótulos verdadeiros para cada amostra

s0 = np.full(40,0)
s1 = np.full(40,1)
s112 = np.full(40,2)
s151 = np.full(40,3)
s162 = np.full(40,4)

verdade = np.concatenate((s0,s1,s112,s151,s162))

4.4 **Métricas** de Desempenho

In [ ]:
print(classification_report(verdade, predito, labels=[0,1,2,3,4]))

4.5 Matriz de Confusão (*confusion matrix*, CM)

In [ ]:
cm = confusion_matrix(verdade, predito, labels=[0,1,2,3,4])

4.6 Visualização da matriz de confusão

In [ ]:
cm = cm[::-1]
x = ['s0', 's1', 's112', 's151', 's162']
y = ['s0', 's1', 's112', 's151', 's162']

cm_text = [[str(y) for y in x] for x in cm]

In [ ]:
fig = px.imshow(cm, x=x, y=y, color_continuous_scale='Viridis', aspect="auto")

fig.update_traces(text=cm_text, texttemplate="%{text}")
fig.update_xaxes(side="top")
fig.update_layout(width = 450, height = 450)
fig.show()



---

